## 조정 결정 계수

선형회귀모형에서 독립변수가 추가되면 결정계수의 값은 항상 증가한다.

In [22]:
import statsmodels.api as sm
from sklearn.datasets import load_boston

boston = load_boston()
dfX0_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
dfX_boston = sm.add_constant(dfX0_boston)
df_boston = pd.concat([dfX_boston, dfy_boston], axis=1)

실제 Boston 데이터를 이용해 일부 피처를 사용한 reduced model과 전체 피처를 사용한 full model 을 이용한 회귀분석을 두번 수행하고 두 모델의 결정계수를 비교해보자.

일부 피처를 사용했을 때의 $R^2$값은 0.735이다.

In [23]:
model_reduced = sm.OLS.from_formula(
    "MEDV ~ ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", data=df_boston)
result = model_reduced.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     113.9
Date:                Mon, 23 Aug 2021   Prob (F-statistic):          1.23e-133
Time:                        11:59:28   Log-Likelihood:                -1504.3
No. Observations:                 506   AIC:                             3035.
Df Residuals:                     493   BIC:                             3090.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.3797      5.143      6.879      0.0

전체 피처를 사용했을 때의 $R^2$는 0.741이다.

In [25]:
model_full  = sm.OLS.from_formula(
    "MEDV ~ CRIM + ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", data=df_boston)
result = model_full .fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 23 Aug 2021   Prob (F-statistic):          6.72e-135
Time:                        12:00:05   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

이처럼 선형 회귀 모형에서 독립변수가 추가되면 결정계수가 항상 증가하는 이유는 다음과 같다.

$$R^2_{Xz} = R^2_{X} + (1-R^2_{X})r^{\ast 2}_{yz}$$

기존의 독립변수 $X$와 새로운 독립변수 $z$를 이용한 경우의 결정계수는 기존의 독립변수 $X$만을 이용했을 때의 결정계수에 $(1 - R^2_X)r_{yz}^2$이라는 항이 더해지게 된다.

여기서 $r^2_{yz}$는 새로운 독립변수 $z$와 종속변수 $y$의 상관계수를 의미하는데 실질적으로 아무런 관계가 없는 데이터라도 상관계수가 0이 나오는 경우는 없기에 모든 항은 양수값을 가지게 된다.

따라서 기존의 독립변수 $X$ 만을 이용한 모델에 새로운 독립변수가 추가되면 상관계수는 반드시 증가하게 되어있다.

따라서 이러한 결정계수의 성질로 인한 독립변수 추가 효과를 상쇄시키기 위한 방법 중 하나로 독립변수의 갯수 $K$에 따라 결정계수의 값을 조정하는 조정 결정 계수를 사용할 수 있다.

$$R_{adj}^2 = 1 - \frac{n-1}{n-K}(1-R^2) = \dfrac{(n-1)R^2 +1-K}{n-K}$$

즉, 독립변수가 많을수록 일종의 패널티를 부여하는 값이 된다.

위 report 결과에서 $Adj. R-squared$에 해당하는 값이 바로 조정 결정계수이다.